In [4]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
#Import packages for this bih

In [12]:
cap = cv2.VideoCapture('pitching videos/overhand 1.MP4') #Read in pitch from folder
frame_list = []  #List where we will store each frame of the video
while(cap.isOpened()): 
      
# Capture frame-by-frame 
    ret, frame = cap.read() 
    if ret == True: 
    # Display the resulting frame 
        cv2.imshow('Frame', frame) 
        frame_list.append(frame)
    # Press Q on keyboard to exit 
        if cv2.waitKey(25) & 0xFF == ord('q'): 
            break
  

    else: 
        break

# the video capture object 
cap.release() 
  
# Closes all the frames 
cv2.destroyAllWindows() 

In [20]:
cv2.imshow('Frame',frame_list[200]) #Show random frame of the video. I picked 200 to do my editing on but it doesn't matter
print(np.shape(frame_list[200].swapaxes(0,1)))
cv2.waitKey(0) 
  
# closing all open windows 
cv2.destroyAllWindows() 



(1920, 1080, 3)


In [21]:
CANVAS_SIZE = (1080,1920)

FINAL_LINE_COLOR = (255, 255, 255)
WORKING_LINE_COLOR = (127, 127, 127)

# ============================================================================

class PolygonDrawer(object):
    def __init__(self, window_name,canvas):
        self.window_name = window_name # Name for our window
        self.canvas = np.copy(canvas)
        self.done = False # Flag signalling we're done
        self.current = (0, 0) # Current position, so we can draw the line-in-progress
        self.points = [] # List of points defining our polygon
#         self.working_points = [] #Points that will show where the cursor currently is, but will not be saved into final image

    def on_mouse(self, event, x, y, buttons, user_param):
        # Mouse callback that gets called for every mouse event (i.e. moving, clicking, etc.)

        if self.done: # Nothing more to do
            return

        if event == cv2.EVENT_MOUSEMOVE:
            # We want to be able to draw the line-in-progress, so update current mouse position
            self.current = (x, y)
        elif event == cv2.EVENT_LBUTTONDOWN:
            # Left click means adding a point at current position to the list of points
            print("Adding point #%d with position(%d,%d)" % (len(self.points), x, y))
            self.points.append((x, y))
        elif event == cv2.EVENT_RBUTTONDOWN:
            # Right click means we're done
            print("Completing polygon with %d points." % len(self.points))
            self.done = True


    def run(self):
        # Let's create our working window and set a mouse callback to handle events
        cv2.namedWindow(self.window_name, flags=cv2.WINDOW_NORMAL)
        cv2.imshow(self.window_name, self.canvas)
        cv2.waitKey(1)
        cv2.setMouseCallback(self.window_name, self.on_mouse)
        clean = np.copy(self.canvas)
        while(not self.done):
            # This is our drawing loop, we just continuously draw new images
            # and show them in the named window
            
                                   
            if (len(self.points) > 0):
                # Draw all the current polygon segments
                cv2.polylines(self.canvas, np.array([self.points]), False, FINAL_LINE_COLOR, 1)
                # And  also show what the current segment would look like
#                 cv2.line(self.canvas, self.points[-1], self.current, WORKING_LINE_COLOR,2)
            # Update the window
            cv2.imshow(self.window_name, self.canvas)
            # And wait 50ms before next iteration (this will pump window messages meanwhile)
            if cv2.waitKey(50) == 27: # ESC hit
                self.done = True

        # User finised entering the polygon points, so let's make the final drawing
        self.canvas = np.copy(clean) #Clean copy of the canvas (no lines) which will get returned to the user
#         of a filled polygon
        if (len(self.points) > 0):
            cv2.polylines(self.canvas, np.array([self.points]),False, FINAL_LINE_COLOR)
# #         And show it
        cv2.imshow(self.window_name, self.canvas)
        # Waiting for the user to press any key
        cv2.waitKey()

        cv2.destroyWindow(self.window_name)
        
        zone = self.canvas - clean #Subtract off difference between original image and marked up image
        
        zone_r = zone[:,:,0] #break into 3 color components r,g,b
        zone_g = zone[:,:,1]
        zone_b = zone[:,:,2]
        

        zone_r[zone_r!=0] = FINAL_LINE_COLOR[0] #Find where these are not equal to zero and set them to the color of the line
                                                #used for marking up
        zone_g[zone_g!=0] = FINAL_LINE_COLOR[1]
        zone_b[zone_b!=0] = FINAL_LINE_COLOR[2]
        
        zone_line = np.stack((zone_r,zone_b,zone_g)).T.swapaxes(0,1) #stack em back up
        
        return self.canvas,zone_line


In [22]:
if __name__ == "__main__":
    canvas = np.copy(frame_list[200])
    pd = PolygonDrawer("Polygon",canvas)
    image,zone_outline = pd.run()
    cv2.imwrite("polygon.png", image)
    print("Polygon = %s" % pd.points)


Adding point #0 with position(1027,739)
Adding point #1 with position(1025,497)
Adding point #2 with position(1280,497)
Adding point #3 with position(1284,754)
Adding point #4 with position(1028,737)
Polygon = [(1027, 739), (1025, 497), (1280, 497), (1284, 754), (1028, 737)]


In [23]:
cv2.imshow('image',canvas) #show original
cv2.waitKey(0) 
  
# closing all open windows 
cv2.destroyAllWindows() 

In [24]:
cv2.imshow('image',image)#show marked up
cv2.waitKey(0) 
  
# closing all open windows 
cv2.destroyAllWindows() 

In [25]:
cv2.imshow('zone',zone_outline) #Show outline drawn by user
cv2.waitKey(0) 
  
# closing all open windows 
cv2.destroyAllWindows() 

print(np.shape(zone_outline))

(1080, 1920, 3)


In [26]:
zone_r = zone_outline[:,:,0] #break up zone into r,g,b
zone_g = zone_outline[:,:,1]
zone_b = zone_outline[:,:,2]
new_frames = [] #new frames for the edited video will be stored here
for k in range(len(frame_list)):
    new_frames.append(frame_list[k])
    current_frame = new_frames[k] #select current frame
    current_frame_r = current_frame[:,:,0] #break into color components
    current_frame_g = current_frame[:,:,1]
    current_frame_b = current_frame[:,:,2]
    
    current_frame_r[zone_r != 0] = np.max(zone_r) #Find on the current frame where the zone_outline is non-zero and set to 
                                                #the max for that color channel
    current_frame_g[zone_g != 0] = np.max(zone_g)
    current_frame_b[zone_b != 0] = np.max(zone_b)
    


In [27]:
# newframes = []
#Write video to output
writer = cv2.VideoWriter("outputvideo.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 120, (1920,1080))

for i in range(len(new_frames)):

    writer.write(new_frames[i])  # write frame into output vid

writer.release()